<a href="https://colab.research.google.com/github/zenon10/POC-OCR/blob/main/clause_label_tagging_vRG20230302.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 KB 5.8 MB/s eta 0:00:00


In [14]:
import warnings
warnings.simplefilter("ignore")

import pandas as pd 
from unidecode import unidecode
import numpy as np

In [6]:
def excel_keyword_parser( keyword_excel):
    '''
    Make dict for all keywords with respected clause label
    input : keyword excel
    output : return dict
    '''
    df = pd.read_excel(keyword_excel,sheet_name = 'Clauses')
    df_fr = df[['Clause Type in English','Keywords in Native Language']].copy()
    df_fr.iloc[:,1] = df_fr.iloc[:,1].str.replace(';',',').replace(['"','-'],' ',regex= True).str.split(',')
    clause_kw_dict = dict(list(zip(df_fr.iloc[:,0],df_fr.iloc[:,1])))
    return clause_kw_dict

In [7]:
#for matching 1 word
def match_word(word,clause):
    '''
    Match word with clause
    input : word from list of keyword and clause
    output : flag if keyword match
    '''
    count = 0
    if '&' in word:
        words = word.split('&')
        for j in words: 
            if clause.__contains__(j):
                count = count + 1
        flag = word if count == len(words) else None
    else:
        flag = word if clause.__contains__(word) else None
    return flag


#Matching a keywords list from clause 
def match_keyword_to_clause(keyword_list,clause):
    '''
    Match list of keywords with clause
    input : list of keyword and clause
    output : return sum of keyword present
    '''
    keyword = [unidecode(j.strip().lower()) for j in keyword_list]
    clause = unidecode(clause.lower().replace('-',' '))
    clause = unidecode(clause.lower().replace('\n',' '))
    cleaned_clause = ' '.join(clause.splitlines())
    key_match = []
    for i in keyword:
        key_match.append(match_word(i,cleaned_clause))
    match_key = list(filter(None.__ne__, key_match))
    return match_key

def para_clause_tagging(clause, clause_kw_dict):
    '''
    Match clause with keyword
    input : clause and dict of keyword
    output : return clause label with max keyword
    '''
    d = dict((k, match_keyword_to_clause(v,clause)) for k, v in clause_kw_dict.items())
    max_length = 0
    max_key_value = None

    for key, value in d.items():
        if isinstance(value, list) and len(value) > max_length:
            max_length = len(value)
            max_key_value = (key, value)

    return max_key_value     

In [87]:
def get_clause_type(excel_file,clause_kw_dict):
    df1 = pd.read_excel(excel_file)
    df1['section_id'].fillna('s0',inplace=True)

    #gives paragraph level clause type
    df1['para_level_clause_type'] = df1.iloc[:,1].apply(lambda x: para_clause_tagging(str(x),clause_kw_dict)[0] if para_clause_tagging(str(x),clause_kw_dict) is not None else None)
    df1['para_level_clause_type_keyword'] = df1.iloc[:,1].apply(lambda x: para_clause_tagging(str(x),clause_kw_dict)[1] if para_clause_tagging(str(x),clause_kw_dict) is not None else None)

    #combine text with identical section id
    sec1 = df1.groupby('section_id')['paragraph'].apply(lambda x: (' '.join(x))).reset_index()
 
    sec1['section_level_clause_type'] = sec1['paragraph'].apply(lambda x: para_clause_tagging(x,clause_kw_dict)[0] if para_clause_tagging(x,clause_kw_dict) is not None else None)
    sec1['section_level_clause_type_keyword'] = sec1['paragraph'].apply(lambda x: para_clause_tagging(x,clause_kw_dict)[1] if para_clause_tagging(x,clause_kw_dict) is not None else None)
    df_sec = pd.merge(df1,sec1.drop('paragraph', axis =1),on =['section_id'],how ='outer') 
    
    df_ab = df_sec.dropna(subset=['section_level_clause_type'], how='all')
    section = df_ab['section_id'].unique().tolist()
    result = pd.DataFrame(columns = ['section_id','clause_type','para_id','clause'])
    for i in section:
        test = df_ab[df_ab['section_id']==i]
        if test['para_level_clause_type'].nunique() ==1:
            if test['para_level_clause_type'].dropna().unique() == test['section_level_clause_type'].unique():
                id = test['para_id'].tolist()
                section_id = i
                clause = " ".join(test['paragraph'])
                clause_type = test['section_level_clause_type'].unique()
                result.loc[len(result.index)] = [section_id,clause_type,id,clause]
        else:
            test.dropna(subset=['para_level_clause_type'],how='all',inplace = True)
            test['kw_str'] = test.para_level_clause_type_keyword.apply(lambda x: ''.join(x))
            test['cond_match'] = np.where(test.para_level_clause_type != test.section_level_clause_type, np.where(test.kw_str.str.find('&'or ' ') ==-1,None, "keep"), "keep")
            test.dropna(subset=['cond_match'],how='all',inplace = True)
            g = test.groupby(['section_id','para_level_clause_type']).agg({'para_id':lambda x: x.tolist(),'paragraph': lambda x:' '.join(x)})
            g.reset_index(inplace= True)
            result = pd.concat([result,g.rename(columns={'paragraph':'clause','para_level_clause_type':'clause_type'})])
    result.to_excel("clause_with_label1.xlsx")

In [10]:
clause_kw_dict = excel_keyword_parser("Clause Management_Deep Dive France_Mapping Document.xlsx")

In [88]:
get_clause_type('s04_cctp_montpellier v2_ocr1.xlsx',clause_kw_dict)

In [82]:
df_ab = ab.dropna(subset=['section_level_clause_type'], how='all')
section = df_ab['section_id'].unique().tolist()
result = pd.DataFrame(columns = ['section_id','clause_type','para_id','clause'])
for i in section:
  test = df_ab[df_ab['section_id']==i]
  if test['para_level_clause_type'].nunique() ==1:
    if test['para_level_clause_type'].dropna().unique() == test['section_level_clause_type'].unique():
      id = test['para_id'].tolist()
      section_id = i
      clause = " ".join(test['paragraph'])
      clause_type = test['section_level_clause_type'].unique()
      result.loc[len(result.index)] = [section_id,clause_type,id,clause]
  else:
    test.dropna(subset=['para_level_clause_type'],how='all',inplace = True)
    test['kw_str'] = test.para_level_clause_type_keyword.apply(lambda x: ''.join(x))
    test['cond_match'] = np.where(test.para_level_clause_type != test.section_level_clause_type, np.where(test.kw_str.str.find('&'or ' ') ==-1,None, "keep"), "keep")
    test.dropna(subset=['cond_match'],how='all',inplace = True)
    g = test.groupby(['section_id','para_level_clause_type']).agg({'para_id':lambda x: x.tolist(),'paragraph': lambda x:' '.join(x)})
    g.reset_index(inplace= True)
    result = pd.concat([result,g.rename(columns={'paragraph':'clause','para_level_clause_type':'clause_type'})])

In [84]:
result

,section_id,clause_type,para_id,clause
0,s1,[Shipment And Delivery Requirements],"[8, 9]",Contenu\nArticle.1 OBJET DU MARCHE... iisisiis...
1,s2,[Tender Description],"[10, 11, 12]",Page 2/9\nCCTP\nSecteur juridique le 08/09/202...
0,s4,Shipment And Delivery Requirements,[23],L’allotissement proposé ne conditionne pas les...
1,s4,Type of quantities,"[28, 29]",3.3 Spécifications quantitatives Les quantités...
0,s5,Product Codification,"[73, 81, 82, 83]",4.5 Codification -symbolisation des produits- ...
1,s5,Shelf Life Requirements,"[90, 96, 97, 98]",Le fabricant ou immatriculant doit s'engager à...
6,s6,[Shipment And Delivery Requirements],"[109, 110, 111, 112]",ARTICLE.6 DEPOT / PRET TEMPORAIRE ET GESTION D...
7,s7,[Samples],"[113, 114, 115, 116]",ARTICLE.7 ESSAIS / DEMANDE DE REFERENCEMENT DE...


In [49]:
test = df_ab[df_ab['section_id']==section[2]]
test.dropna(subset=['para_level_clause_type'],how='all',inplace = True)
test1 = test[test.para_level_clause_type != test.section_level_clause_type]
test2 = test[test.para_level_clause_type == test.section_level_clause_type]


,para_id,paragraph,is_article_ind,article_id,Is_section_ind,is_article_ind_nw_logic,section_id,para_level_clause_type,para_level_clause_type_keyword,section_level_clause_type,section_level_clause_type_keyword
23,23,L’allotissement proposé ne conditionne pas les...,0,a2,0,0,s4,Shipment And Delivery Requirements,[mise&en&depot],Type of quantities,"[quantite&indiquee, quantite&previsionnelle, s..."
26,26,Page 3/9\nCCTP\nSecteur juridique le 08/09/202...,0,a2,0,0,s4,Samples,"[essai, essais]",Type of quantities,"[quantite&indiquee, quantite&previsionnelle, s..."
36,36,Le titulaire du marché s'engage à assurer la f...,0,a2,0,0,s4,Samples,[essai],Type of quantities,"[quantite&indiquee, quantite&previsionnelle, s..."


In [24]:
test = df_ab[df_ab['section_id']==section[2]]
test = test[test.para_level_clause_type != test.section_level_clause_type]
test.dropna(subset=['para_level_clause_type'],how='all',inplace = True)
test['kw_str'] = test.para_level_clause_type_keyword.apply(lambda x: ''.join(x))
test['cond_match'] = np.where(test.para_level_clause_type != test.section_level_clause_type, np.where(test.kw_str.str.find('&') ==-1,None, "keep"), "keep")
test.dropna(subset=['cond_match'],how='all',inplace = True)
g = test.groupby(['section_id','para_level_clause_type']).agg({'para_id':lambda x: x.tolist(),'paragraph': lambda x:' '.join(x)})
g.reset_index(inplace= True)
b = g[['para_id','paragraph','section_id','para_level_clause_type']]

In [25]:
b


,para_id,paragraph,section_id,para_level_clause_type
0,[23],L’allotissement proposé ne conditionne pas les...,s4,Shipment And Delivery Requirements
1,"[28, 29]",3.3 Spécifications quantitatives Les quantités...,s4,Type of quantities


In [1]:
g = test.groupby(['section_id','para_level_clause_type']).agg({'para_id':lambda x: x.tolist(),'paragraph': lambda x:' '.join(x)})
b = g['para_id','paragraph','section_id','para_level_clause_type']

NameError: ignored

## rough

In [29]:
# Suppose you have a dataset stored in a Pandas DataFrame
import pandas as pd

df = pd.DataFrame({'Column1': [1, 2, 3, 4, 5],
                   'Column2': [1, 1, 2, 2, 3],
                   'Column3': ['Text1', 'Text2', 'Text3', 'Text4', 'Text5'],
                   'Section_id': ['A', 'B', 'A', 'C', 'B']})

# Create an empty dictionary to store the combined texts
combined_text = {}
df

,Column1,Column2,Column3,Section_id
0,1,1,Text1,A
1,2,1,Text2,B
2,3,2,Text3,A
3,4,2,Text4,C
4,5,3,Text5,B


In [30]:
# Iterate over each row of the dataset
for index, row in df.iterrows():
    # Check if the values in Column1, Column2, and Section_id are the same as any previous row
    if (row['Column1'], row['Column2'], row['Section_id']) in combined_text:
        # If the values match, append the text in Column3 to the previous row's text
        combined_text[(row['Column1'], row['Column2'], row['Section_id'])] += ' ' + row['Column3']
    else:
        # If the values don't match, add a new key-value pair to the dictionary
        combined_text[(row['Column1'], row['Column2'], row['Section_id'])] = row['Column3']
# Convert the dictionary back to a DataFrame
combined_df = pd.DataFrame(list(combined_text.items()), columns=['Column1', 'Column2', 'Section_id', 'Column3'])

# Reorder the columns in the DataFrame
combined_df = combined_df[['Column1', 'Column2', 'Column3', 'Section_id']]

# Sort the DataFrame by Column1 and Column2
combined_df.sort_values(by=['Column1', 'Column2'], inplace=True)

# Reset the index of the DataFrame
combined_df.reset_index(drop=True, inplace=True)

ValueError: ignored

In [26]:
combined_text

{(1, 1, 'A'): 'Text1',
 (2, 1, 'B'): 'Text2',
 (3, 2, 'A'): 'Text3',
 (4, 2, 'C'): 'Text4',
 (5, 3, 'B'): 'Text5'}